In [ ]:
import smll
import time
import zstd
import zlib
import bz2
import lzma
import base64

def compress_and_compare(
    input: str,
):
    print("Compressing the following input:\n")
    print(input)

    with smll.Compressor.from_pretrained(repo_id="QuantFactory/SmolLM2-360M-GGUF",filename="*Q4_0.gguf") as compressor:

        start = time.time()
        compressed = compressor.compress(input)
        end = time.time()
        print(f"\nCompressed in {end - start:.2f} seconds.\n")
        # compare results
        original_bytes = input.encode("utf-8")
        original_size = len(original_bytes)

        # calc llm compression ratio
        compressed_size = len(compressed)
        compression_ratio = original_size / compressed_size

        # calc compression algorithms
        gzip_size = len(zlib.compress(original_bytes, level=9))
        bz2_size = len(bz2.compress(original_bytes, compresslevel=9))
        lzma_size = len(lzma.compress(original_bytes, preset=9))
        zstd_size = len(zstd.compress(original_bytes, 22))

        print(f"Encoded: {base64.b64encode(compressed).decode('ascii')}")
        print("\nCompression Results:")
        print(f"  Original:        {original_size:>6} bytes")
        print(f"  LLM compressed:  {compressed_size:>6} bytes ({compression_ratio:.2f}x)")
        print(
            f"  ZSTD (level 22):  {zstd_size:>6} bytes ({original_size / zstd_size:.2f}x)"
        )
        print(f"  GZIP (level 9):  {gzip_size:>6} bytes ({original_size / gzip_size:.2f}x)")
        print(f"  BZ2 (level 9):   {bz2_size:>6} bytes ({original_size / bz2_size:.2f}x)")
        print(f"  LZMA (level 9):  {lzma_size:>6} bytes ({original_size / lzma_size:.2f}x)")

        print("Decompressing...")

        start: float = time.time()
        decompressed = compressor.decompress(compressed)
        print(decompressed)
        end = time.time()
        print(f"\nDecompressed in {end - start:.2f} seconds.\n")

        assert decompressed == input, "Decompression failed!"
        print("\nDecompression successful!")

In [3]:
# compress text & compare against other algs
input_text = """"/**
 * Copyright (c) Meta Platforms, Inc. and affiliates.
 *
 * This source code is licensed under the MIT license found in the
 * LICENSE file in the root directory of this source tree.
 *
 * @flow
 */

// Keep in sync with ReactServerConsoleConfig
const badgeFormat = '%c%s%c';
// Same badge styling as DevTools.
const badgeStyle =
  // We use a fixed background if light-dark is not supported, otherwise
  // we use a transparent background.
  'background: #e6e6e6;' +
  'background: light-dark(rgba(0,0,0,0.1), rgba(255,255,255,0.25));' +
  'color: #000000;' +
  'color: light-dark(#000000, #ffffff);' +
  'border-radius: 2px';
const resetStyle = '';
const pad = ' ';

const bind = Function.prototype.bind;

export function bindToConsole(
  methodName: string,
  args: Array<any>,
  badgeName: string,
): () => any {
  let offset = 0;
  switch (methodName) {
    case 'dir':
    case 'dirxml':
    case 'groupEnd':
    case 'table': {
      // These methods cannot be colorized because they don't take a formatting string.
      // $FlowFixMe
      return bind.apply(console[methodName], [console].concat(args)); // eslint-disable-line react-internal/no-production-logging
    }
    case 'assert': {
      // assert takes formatting options as the second argument.
      offset = 1;
    }
  }

  const newArgs = args.slice(0);
  if (typeof newArgs[offset] === 'string') {
    newArgs.splice(
      offset,
      1,
      badgeFormat + ' ' + newArgs[offset],
      badgeStyle,
      pad + badgeName + pad,
      resetStyle,
    );
  } else {
    newArgs.splice(
      offset,
      0,
      badgeFormat,
      badgeStyle,
      pad + badgeName + pad,
      resetStyle,
    );
  }

  // The "this" binding in the "bind";
  newArgs.unshift(console);

  // $FlowFixMe
  return bind.apply(console[methodName], newArgs); // eslint-disable-line react-internal/no-production-logging
}"""

compress_and_compare(input_text)

Compressing the following input:

"/**
 * Copyright (c) Meta Platforms, Inc. and affiliates.
 *
 * This source code is licensed under the MIT license found in the
 * LICENSE file in the root directory of this source tree.
 *
 * @flow
 */

// Keep in sync with ReactServerConsoleConfig
const badgeFormat = '%c%s%c';
// Same badge styling as DevTools.
const badgeStyle =
  // We use a fixed background if light-dark is not supported, otherwise
  // we use a transparent background.
  'background: #e6e6e6;' +
  'background: light-dark(rgba(0,0,0,0.1), rgba(255,255,255,0.25));' +
  'color: #000000;' +
  'color: light-dark(#000000, #ffffff);' +
  'border-radius: 2px';
const resetStyle = '';
const pad = ' ';

const bind = Function.prototype.bind;

export function bindToConsole(
  methodName: string,
  args: Array<any>,
  badgeName: string,
): () => any {
  let offset = 0;
  switch (methodName) {
    case 'dir':
    case 'dirxml':
    case 'groupEnd':
    case 'table': {
      // These methods can

AssertionError: Decompression failed!